Alternate functions that could be used in menu

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
import py4j

spark = SparkSession\
    .builder\
    .config("spark.jars", "/Users/nahidrahman/opt/miniconda3/lib/python3.9/site-packages/pyspark/jars/mysql-connector-j-8.0.31.jar")\
    .appName("menu")\
    .getOrCreate()

df_branch = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_BRANCH") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_credit = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_custmer = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTMER") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [7]:
year = input("Choose the year:")
month = input("Choose the month:")
zipcode = input("Choose zipcode:")
#function that displays the transactions made by customers living in a given zip code for a given month and year, in descending order.
def transactions(year, month, zipcode):
    df = df_credit.join(df_custmer, df_credit.CUST_SSN == df_custmer.SSN,  'outer')
    df.filter( (df['year'] == year) & (df['month'] == month) & (df['CUST_ZIP'] == zipcode)).sort('day', ascending= False).show(10)        
transactions(year, month, zipcode)
print("...")

+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+------+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
|BRANCH_CODE|CREDIT_CARD_NO|CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|TIMEID|APT_NO|CREDIT_CARD_NO|CUST_CITY|CUST_COUNTRY|CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED|MIDDLE_NAME|SSN|STREET_NAME|
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+------+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+------+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------

In [9]:
from pyspark.sql.functions import round

print ("Gas\nEntertainment\nHealthcare\nGrocery\nTest\nEducation")
types = input("Enter one of the following transaction types above: ") 
print('...')

#function that displays the number and total values of transactions for a given type.
#Used credit pyspark dataframe and filtered based on transaction type. To get the total values, used the group by and sum method.
def total_transactions(type):
    transactions_of_type = df_credit.filter(df_credit['transaction_type'] == type).count()
    print(f'The number of {type} transactions is {transactions_of_type}')
    
    value_of_transaction_type = df_credit.filter(df_credit['transaction_type'] == type).groupBy().sum('transaction_value')
    formatted_total_value = value_of_transaction_type.select(round(value_of_transaction_type[0], 2)).collect()[0][0]

    print(f"The value of these transactions is ${formatted_total_value}")

total_transactions(types)
print("...")


Gas
Entertainment
Healthcare
Grocery
Test
Education
...
The number of Gas transactions is 6605
The value of these transactions is $336059.26
...


In [10]:
from pyspark.sql.functions import round

state = input("Enter a state (abbreviation): ")
print('...')

def branch_transactions(state):
    df2 = df_credit.join(df_branch, df_credit.BRANCH_CODE == df_branch.BRANCH_CODE, 'outer')
    transactions_in_state = df2.filter(df2['BRANCH_STATE'] == state).count()
    print(f"The number of transactions in {state} is {transactions_in_state}")
    value_of_transactions = df2.filter(df2['BRANCH_STATE'] == state).groupBy().sum('transaction_value')
    formatted_total_value = value_of_transactions.select(round(value_of_transactions[0], 2)).collect()[0][0]
    print(f"The value of these transactions is ${formatted_total_value}")

branch_transactions(state)
print('...')

...
The number of transactions in NY is 4143
The value of these transactions is $213717.38
...


In [19]:
SSN = input("Enter your SSN:")

#function that views the existing account details of a customer given SSN.
def account_details(SSN):
    df_custmer.filter(df_custmer['SSN'] == SSN).show()
account_details(SSN)

+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
|APT_NO|CREDIT_CARD_NO|CUST_CITY|CUST_COUNTRY|CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED|MIDDLE_NAME|SSN|STREET_NAME|
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+



In [ ]:
from pyspark.sql.functions import round

credit_card_no = input("Enter your credit card number: ")
month = input("Enter a month: ")
year = input("Enter a year: ")

# Function that generates a monthly bill for a credit card number for a given month and year.
import calendar

def monthly_bill(credit_card_no, month, year):
    month = int(month)
    
    if 1 <= month <= 12:
        bill = df_credit.filter(
            (df_credit['credit_card_no'] == credit_card_no) &
            (df_credit['month'] == month) &
            (df_credit['year'] == year)
        ).toPandas()

        bill_sum = bill['TRANSACTION_VALUE'].sum()

        month_name = calendar.month_name[month]

        # Don't use PySpark's round function here
        formatted_bill_sum = round(bill_sum, 2)

        print(f"The monthly bill for {month_name} is ${formatted_bill_sum}")
    else:
        print("Invalid month. Please enter a month between 1 and 12.")

monthly_bill(credit_card_no, month, year)
